In [1]:
import geopandas as gpd
import folium
from shapely.geometry import Point
import pandas as pd
import numpy as np

In [2]:
from shapely.geometry import Point

In [23]:
# 读取公园数据
park_gdf = gpd.read_file('SeattleParkPoints.geojson')
boundary_gdf = gpd.read_file('Seattle_City_Boundary.geojson')

# 确保边界和数据使用相同的坐标系统
if park_gdf.crs != boundary_gdf.crs:
    park_gdf = park_gdf.to_crs(boundary_gdf.crs)

park_gdf.head()

,OBJECTID,PROPNAME,ADDRESS,DIVISION,ADA,GIS_EDT_DT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,...,LOCID,AMWO_ID,RES1,RES2,GLOBALID,GIS_CREATOR,GIS_CRT_DT,GIS_EDITOR,SDQL,geometry
0,7044,Alki Playground,5817 SW Lander St,South,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.577899,-122.407731,1.251850e+06,214775.000001,...,1049,PLAY-ALKIPG,None,None,17ca3d9c-c23f-41f4-927f-5f4583822a15,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (1251850.25 214775)
1,7045,B.F. Day Playground,4020 Fremont Ave N,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.655668,-122.348906,1.266925e+06,242845.296892,...,2552,PLAY-DAYPG,None,None,2b87dea1-038d-4a59-98f0-db9ad09725ff,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (1266925.25 242845.297)
2,7046,Ballard Playground,2644 NW 60th St,North,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.672648,-122.392548,1.256297e+06,249252.812470,...,1108,PLAY-BLPG,None,None,ff837c9b-46fd-4b79-8eda-d51a40268425,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (1256297 249252.812)
3,7047,Bayview Playground,2614 24th Ave W,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.644301,-122.386906,1.257478e+06,238886.812472,...,1110,PLAY-BVPG,None,None,c7a03c40-dbb7-4dcf-ac40-bb26e9c9b3b2,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (1257478.25 238886.812)
4,7048,Bayview-Kinnear Park,3rd Ave W / W Prospect St,Central,Yes,"Mon, 29 Jul 2024 19:16:43 GMT",47.628900,-122.360237,1.263939e+06,233138.296906,...,2374,PLAY-KERPK,None,None,10ab7d68-dd5c-43cc-98a4-b85c233bdc56,SeattleParks_SeattleCityGIS,"Mon, 29 Jul 2024 19:16:43 GMT",SeattleParks_SeattleCityGIS,QL-D3,POINT (1263939.375 233138.297)


In [5]:

if park_gdf.crs != 'EPSG:4326':
    park_gdf = park_gdf.to_crs(epsg=3857)  # Changed from park_gdf.crs(epsg=3857) to park_gdf.to_crs(epsg=3857)

centroid = park_gdf.geometry.centroid
center_x = centroid.x.mean()
center_y = centroid.y.mean()

# 将计算出的中心点转换回 WGS84 坐标系统
center_point = gpd.GeoSeries([Point(center_x, center_y)], crs='EPSG:3857').to_crs(epsg=4326).geometry.iloc[0]
center_location = [center_point.y, center_point.x]

In [6]:
# 定义样式函数
def style_function(feature):
    return {
        'color': 'blue',    # 线条颜色
        'weight': 3,        # 线条粗细
        'opacity': 0.7      # 线条透明度
    }

# 初始化 Folium 地图
m = folium.Map(location=center_location, zoom_start=12)

# 添加一个TileLayer图层
folium.TileLayer(tiles ='cartodbpositron').add_to(m)

# 将裁剪后的 GeoDataFrame 转换为 GeoJSON 并添加到地图上
folium.GeoJson(park_gdf, style_function=style_function).add_to(m)

# 保存或展示地图
m.save('Park_Markers_Seattle.html')
m

In [26]:
park = park_gdf.copy()

In [35]:
park = park.dropna(subset = ['LONGITUDE','LATITUDE'])
park[['LONGITUDE','LATITUDE']].isna().sum()

LONGITUDE    0
LATITUDE     0
dtype: int64

In [24]:
park_gdf.columns

Index(['OBJECTID', 'PROPNAME', 'ADDRESS', 'DIVISION', 'ADA', 'GIS_EDT_DT',
       'LATITUDE', 'LONGITUDE', 'POINT_X', 'POINT_Y', 'PMAID', 'LOCID',
       'AMWO_ID', 'RES1', 'RES2', 'GLOBALID', 'GIS_CREATOR', 'GIS_CRT_DT',
       'GIS_EDITOR', 'SDQL', 'geometry'],
      dtype='object')

In [25]:
# 初始化 Folium 地图
m = folium.Map(location=center_location, zoom_start=12)

# 添加一个TileLayer图层
folium.TileLayer(tiles ='cartodbpositron').add_to(m)

for index,row in park_gdf.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        loc = [row['LATITUDE'],row['LONGITUDE']]
        tooltip_text = f"park name: {row['PROPNAME']}"
        folium.Circle(
            location=loc, 
            radius=500,  
            color='salmon', 
            fill=True,  
            fill_color='salmon',  
            fill_opacity=0.5,  
            tooltip = tooltip_text
        ).add_to(m)        

# 打印
m.save('Park_Points_Seattle.html')
m